In [73]:
import pandas as pd
import rdkit.Chem as Chem
import numpy as np

In [2]:
db_df = pd.read_csv('drugbank_filtered.csv')
cs_df = pd.read_csv('{"model_family": "jova", "dataset": "egfr_case_study", "cviews": "ecfp8-gconv", "pviews": "rnn-psc", "split": "random", "mode": "case_study", "model_ds": "kiba", "target": "EGFR", "cv": "False", "seeds": "1", "date": "2020_01_27__01_29_45"}.csv')
EGFR_df = pd.read_csv('drugbank_EGFR.csv')
db_df.shape, cs_df.shape, EGFR_df.shape

((9484, 5), (9484, 11), (30, 5))

In [3]:
cs_df.head()

,dataset,y_true,y_pred,view1,entity1,top_segments1,view2,entity2,sequence,top_segments2,top_residues
0,egfr_cs,0.0,3.870844,gconv,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(CSc3nn...,"[['C', 12], ['C', 18], ['O', 9], ['C', 30], ['...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
1,egfr_cs,0.0,3.917361,gconv,CC[Si](C)(C)C,"[['C', 4], ['C', 5], ['C', 1], ['Si', 2], ['C'...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
2,egfr_cs,0.0,3.767989,gconv,O=C(/C=C/c1c(C(=O)O)[nH]c2cc(Cl)cc(Cl)c12)Nc1c...,"[['C', 5], ['O', 8], ['C', 11], ['C', 12], ['C...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
3,egfr_cs,0.0,3.248190,gconv,N=C(O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)...,"[['C', 22], ['O', 16], ['C', 45], ['O', 18], [...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['EDHFLSLQRMFNN', 'PPICTIDVYMIMV', 'TSNKLTQLGT...",['Glu45 Asp46 His47 Phe48 Leu49 Ser50 Leu51 Gl...
4,egfr_cs,0.0,3.496230,gconv,O=C[C@H](CC(=O)O)NC(=O)[C@@H](CCCCNC(=O)c1ccc2...,"[['C', 18], ['O', 6], ['C', 14], ['C', 16], ['...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...


In [5]:
def get_canonical_smiles(smiles):
    can_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles), True)
    return can_smiles

In [7]:
predictions = {'id':[],'drug':[],'target':[], 'value':[], 'drugbank?':[]}
for idx, row in cs_df.iterrows():
    cs_smiles = get_canonical_smiles(row['entity1'])
    db_row = db_df[db_df['canonical_smiles']==cs_smiles]
    db_id = db_row['ID'].iloc[0]
    predictions['id'].append(db_id)
    predictions['drug'].append(db_row['Name'].iloc[0])
    predictions['target'].append(row['entity2'])
    predictions['value'].append(row['y_pred'])
    s = EGFR_df[EGFR_df['DRUGBANK ID']==db_id]
    if len(s) > 0:
        predictions['drugbank?'].append('yes')
    else:
        predictions['drugbank?'].append('no')    

In [39]:
predictions = pd.DataFrame(predictions)
predictions = predictions.sort_values('value', ascending=True).reset_index(drop=True)
predictions.head(50)

,id,drug,target,value,drugbank?
0,DB11963,Dacomitinib,EGFR,1.313624,yes
1,DB06021,AV-412,EGFR,1.516182,yes
2,DB07788,"(3R,5Z,8S,9S,11E)-8,9,16-TRIHYDROXY-14-METHOXY...",EGFR,1.692564,no
3,DB12818,NM-3,EGFR,1.775189,no
4,DB14944,Tarloxotinib,EGFR,1.834283,no
5,DB02848,"{4-[3-(6,7-Diethoxy-Quinazolin-4-Ylamino)-Phen...",EGFR,1.901084,no
6,DB05944,Varlitinib,EGFR,1.912083,yes
7,DB12669,4SC-203,EGFR,1.915282,no
8,DB12114,Poziotinib,EGFR,1.993397,no
9,DB14993,Pyrotinib,EGFR,1.997456,no


In [46]:
predicted_true_dti = pd.DataFrame({k:[] for k in predictions.columns})
for idx, row in predictions.iterrows():
    if row['drugbank?'] == 'yes':
        predicted_true_dti = predicted_true_dti.append(row)
predicted_true_dti.head(50).reset_index()

,index,id,drug,target,value,drugbank?
0,0,DB11963,Dacomitinib,EGFR,1.313624,yes
1,1,DB06021,AV-412,EGFR,1.516182,yes
2,6,DB05944,Varlitinib,EGFR,1.912083,yes
3,26,DB12267,Brigatinib,EGFR,2.716983,yes
4,31,DB15035,Zanubrutinib,EGFR,2.764389,yes
5,45,DB13164,Olmutinib,EGFR,2.878636,yes
6,491,DB07602,S-{3-[(4-ANILINOQUINAZOLIN-6-YL)AMINO]-3-OXOPR...,EGFR,3.470041,yes
7,4967,DB09330,Osimertinib,EGFR,3.898723,yes


In [48]:
predictions.shape

(9484, 5)

In [41]:
predictions.to_csv('predictions_EGFR_kiba.csv', index=False)

In [75]:
k = 50
predictions_topk = predictions.iloc[:k,:]
predictions_topk.to_csv('predictions_topk.csv', index=True)

In [67]:
drugbank_removed_df = pd.read_csv('drugbank_kiba_intersection.csv')
drugbank_removed_df.shape

(1123, 4)

In [44]:
removed_true_dti = pd.DataFrame({'ID':[],'Name':[],'SMILES':[],'canonical_smiles':[]})
for idx, row in drugbank_removed_df.iterrows():
    s = EGFR_df[EGFR_df['DRUGBANK ID']==row['ID']]
    if len(s) > 0:
        removed_true_dti = removed_true_dti.append(row)
removed_true_dti.shape

(13, 4)

In [49]:
removed_true_dti.head(50).reset_index(drop=True)

,ID,Name,SMILES,canonical_smiles
0,DB00281,Lidocaine,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C,CCN(CC)CC(=O)Nc1c(C)cccc1C
1,DB00317,Gefitinib,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1
2,DB00530,Erlotinib,COCCOC1=CC2=C(C=C1OCCOC)C(NC1=CC(=CC=C1)C#C)=N...,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1
3,DB01259,Lapatinib,CS(=O)(=O)CCNCC1=CC=C(O1)C1=CC2=C(C=C1)N=CN=C2...,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...
4,DB03496,Alvocidib,CN1CC[C@@H]([C@H](O)C1)C1=C(O)C=C(O)C2=C1OC(=C...,CN1CC[C@H](c2c(O)cc(O)c3c(=O)cc(-c4ccccc4Cl)oc...
5,DB05294,Vandetanib,COC1=C(OCC2CCN(C)CC2)C=C2N=CN=C(NC3=C(F)C=C(Br...,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
6,DB05424,Canertinib,FC1=C(Cl)C=C(NC2=NC=NC3=CC(OCCCN4CCOCC4)=C(NC(...,C=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OCCCN...
7,DB05524,Pelitinib,CCOC1=CC2=C(C=C1NC(=O)\C=C\CN(C)C)C(NC1=CC(Cl)...,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...
8,DB07662,N-[4-(3-BROMO-PHENYLAMINO)-QUINAZOLIN-6-YL]-AC...,BrC1=CC=CC(NC2=C3C=C(NC(=O)C=C)C=CC3=NC=N2)=C1,C=CC(=O)Nc1ccc2ncnc(Nc3cccc(Br)c3)c2c1
9,DB08916,Afatinib,CN(C)C\C=C\C(=O)NC1=C(O[C@H]2CCOC2)C=C2N=CN=C(...,CN(C)C/C=C/C(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2...


In [50]:
db_canonicalized = pd.read_csv('drugbank_canonicalized.csv')
db_canonicalized.shape

(10607, 4)

In [53]:
db_can_true_dti = pd.DataFrame({'ID':[],'Name':[],'SMILES':[],'canonical_smiles':[]})
for idx, row in db_canonicalized.iterrows():
    s = EGFR_df[EGFR_df['DRUGBANK ID']==row['ID']]
    if len(s) > 0:
        db_can_true_dti = db_can_true_dti.append(row)
db_can_true_dti = db_can_true_dti.reset_index(drop=True)
db_can_true_dti.shape

(21, 4)

In [54]:
db_can_true_dti.head(25)

,ID,Name,SMILES,canonical_smiles
0,DB00281,Lidocaine,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C,CCN(CC)CC(=O)Nc1c(C)cccc1C
1,DB00317,Gefitinib,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1
2,DB00530,Erlotinib,COCCOC1=CC2=C(C=C1OCCOC)C(NC1=CC(=CC=C1)C#C)=N...,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1
3,DB01259,Lapatinib,CS(=O)(=O)CCNCC1=CC=C(O1)C1=CC2=C(C=C1)N=CN=C2...,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...
4,DB03496,Alvocidib,CN1CC[C@@H]([C@H](O)C1)C1=C(O)C=C(O)C2=C1OC(=C...,CN1CC[C@H](c2c(O)cc(O)c3c(=O)cc(-c4ccccc4Cl)oc...
5,DB05294,Vandetanib,COC1=C(OCC2CCN(C)CC2)C=C2N=CN=C(NC3=C(F)C=C(Br...,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
6,DB05424,Canertinib,FC1=C(Cl)C=C(NC2=NC=NC3=CC(OCCCN4CCOCC4)=C(NC(...,C=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1OCCCN...
7,DB05524,Pelitinib,CCOC1=CC2=C(C=C1NC(=O)\C=C\CN(C)C)C(NC1=CC(Cl)...,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...
8,DB05944,Varlitinib,C[C@@H]1COC(NC2=CC=C3N=CN=C(NC4=CC=C(OCC5=NC=C...,C[C@@H]1COC(Nc2ccc3ncnc(Nc4ccc(OCc5nccs5)c(Cl)...
9,DB06021,AV-412,CC1=CC=C(C=C1)S(O)(=O)=O.CC1=CC=C(C=C1)S(O)(=O...,C=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc1C#CC(...


In [55]:
cs_unfiltered_df = pd.read_csv('{"model_family": "jova", "dataset": "egfr_unfiltered", "cviews": "ecfp8-gconv", "pviews": "rnn-psc", "split": "random", "mode": "case_study", "model_ds": "kiba", "target": "EGFR", "cv": "False", "seeds": "1", "date": "2020_01_27__02_29_59"}.csv')
cs_unfiltered_df.shape

(10607, 11)

In [56]:
predictions_unf = {'id':[],'drug':[],'target':[], 'value':[], 'drugbank?':[]}
for idx, row in cs_unfiltered_df.iterrows():
    cs_unf_smiles = get_canonical_smiles(row['entity1'])
    db_row_unf = db_canonicalized[db_canonicalized['canonical_smiles']==cs_unf_smiles]
    db_id_unf = db_row_unf['ID'].iloc[0]
    predictions_unf['id'].append(db_id_unf)
    predictions_unf['drug'].append(db_row_unf['Name'].iloc[0])
    predictions_unf['target'].append(row['entity2'])
    predictions_unf['value'].append(row['y_pred'])
    s = EGFR_df[EGFR_df['DRUGBANK ID']==db_id_unf]
    if len(s) > 0:
        predictions_unf['drugbank?'].append('yes')
    else:
        predictions_unf['drugbank?'].append('no')
predictions_unf = pd.DataFrame(predictions_unf)
predictions_unf = predictions_unf.sort_values('value', ascending=True).reset_index(drop=True)
predictions_unf.head(50)

,id,drug,target,value,drugbank?
0,DB05424,Canertinib,EGFR,0.923217,yes
1,DB08916,Afatinib,EGFR,1.145424,yes
2,DB12174,CUDC-101,EGFR,1.161614,no
3,DB07662,N-[4-(3-BROMO-PHENYLAMINO)-QUINAZOLIN-6-YL]-AC...,EGFR,1.181644,yes
4,DB00317,Gefitinib,EGFR,1.301118,yes
5,DB11963,Dacomitinib,EGFR,1.313624,yes
6,DB08564,(2E)-N-{4-[(3-bromophenyl)amino]quinazolin-6-y...,EGFR,1.371380,no
7,DB00530,Erlotinib,EGFR,1.396310,yes
8,DB03365,"4-[3-Hydroxyanilino]-6,7-Dimethoxyquinazoline",EGFR,1.439483,no
9,DB06021,AV-412,EGFR,1.516182,yes


In [57]:
predicted_unf_true_dti = pd.DataFrame({k:[] for k in predictions_unf.columns})
for idx, row in predictions_unf.iterrows():
    if row['drugbank?'] == 'yes':
        predicted_unf_true_dti = predicted_unf_true_dti.append(row)
predicted_unf_true_dti.head(50).reset_index()

,index,id,drug,target,value,drugbank?
0,0,DB05424,Canertinib,EGFR,0.923217,yes
1,1,DB08916,Afatinib,EGFR,1.145424,yes
2,3,DB07662,N-[4-(3-BROMO-PHENYLAMINO)-QUINAZOLIN-6-YL]-AC...,EGFR,1.181644,yes
3,4,DB00317,Gefitinib,EGFR,1.301118,yes
4,5,DB11963,Dacomitinib,EGFR,1.313624,yes
5,7,DB00530,Erlotinib,EGFR,1.396310,yes
6,9,DB06021,AV-412,EGFR,1.516182,yes
7,11,DB11828,Neratinib,EGFR,1.692170,yes
8,18,DB05944,Varlitinib,EGFR,1.912083,yes
9,20,DB11737,Icotinib,EGFR,1.947831,yes


In [59]:
db_small_mols = pd.read_csv('drugbank_smiles.csv')
db_small_mols.shape

(10630, 3)

In [60]:
db_small_mols_dti = pd.DataFrame({'ID':[],'Name':[],'SMILES':[],'canonical_smiles':[]})
for idx, row in db_small_mols.iterrows():
    s = EGFR_df[EGFR_df['DRUGBANK ID']==row['ID']]
    if len(s) > 0:
        db_small_mols_dti = db_small_mols_dti.append(row)
db_small_mols_dti = db_small_mols_dti.reset_index(drop=True)
db_small_mols_dti.shape

(21, 4)

In [61]:
db_all = pd.read_csv('drugbank_all.csv')
db_all.shape

(13339, 3)

In [62]:
db_all_dti = pd.DataFrame({'ID':[],'Name':[],'SMILES':[],'canonical_smiles':[]})
for idx, row in db_all.iterrows():
    s = EGFR_df[EGFR_df['DRUGBANK ID']==row['ID']]
    if len(s) > 0:
        db_all_dti = db_all_dti.append(row)
db_all_dti = db_all_dti.reset_index(drop=True)
db_all_dti.shape

(30, 4)

In [63]:
db_all_dti

,ID,Name,SMILES,canonical_smiles
0,DB00002,Cetuximab,NaN,NaN
1,DB00072,Trastuzumab,NaN,NaN
2,DB00281,Lidocaine,CCN(CC)CC(=O)NC1=C(C)C=CC=C1C,NaN
3,DB00317,Gefitinib,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...,NaN
4,DB00530,Erlotinib,COCCOC1=CC2=C(C=C1OCCOC)C(NC1=CC(=CC=C1)C#C)=N...,NaN
5,DB01259,Lapatinib,CS(=O)(=O)CCNCC1=CC=C(O1)C1=CC2=C(C=C1)N=CN=C2...,NaN
6,DB01269,Panitumumab,NaN,NaN
7,DB03496,Alvocidib,CN1CC[C@@H]([C@H](O)C1)C1=C(O)C=C(O)C2=C1OC(=C...,NaN
8,DB04988,IGN311,NaN,NaN
9,DB05101,Matuzumab,NaN,NaN


In [79]:
egfr = 'MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPSIATGMVGALLLLLVVALGIGLFMRRRHIVRKRTLRRLLQERELVEPLTPSGEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLPSPTDSNFYRALMDEEDMDDVVDADEYLIPQQGFFSSPSTSRTPLLSSLSATSNNSTVACIDRNGLQSCPIKEDSFLQRYSSDPTGALTEDSIDDTFLPVPEYINQSVPKRPAGSVQNPVYHNQPLNPAPSRDPHYQDPHSTAVGNPEYLNTVQPTCVNSTFDSPAHWAQKGSHQISLDNPDYQQDFFPKEAKPNGIFKGSTAENAEYLRVAPQSSEFIGA'
egfr_1m17 = 'GEAPNQALLRILKETEFKKIKVLGSGAFGTVYKGLWIPEGEKVKIPVAIKELREATSPKANKEILDEAYVMASVDNPHVCRLLGICLTSTVQLITQLMPFGCLLDYVREHKDNIGSQYLLNWCVQIAKGMNYLEDRRLVHRDLAARNVLVKTPQHVKITDFGLAKLLGAEEKEYHAEGGKVPIKWMALESILHRIYTHQSDVWSYGVTVWELMTFGSKPYDGIPASEISSILEKGERLPQPPICTIDVYMIMVKCWMIDADSRPKFRELIIEFSKMARDPQRYLVIQGDERMHLMDEEDMDDVVDADEYLIP'
len(egfr), len(egfr_1m17)

(1210, 312)